In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [ ]:
def get_page(link):
    tries = 0
    while tries < 3:
        try:
            page_link = link
            # fetch the content from url
            page_response = requests.get(page_link, timeout=5)
            # parse html
            page_content = BeautifulSoup(page_response.content, "html.parser")
            break
        except:
            print('Sleeping')
            print(link)
            tries+=1
            time.sleep(180)            
    return page_content

In [ ]:
def collect_data(link):
    #coleta nomes e links para cada país/continente
    try:
        page_content = get_page(link)
        
        player_info = page_content.find('div', {'class': 'info'}).text.split('  ')[0].replace('\n','')
        player_meta = page_content.find('div', {'class': 'meta'}).text.split('  ')[0].replace('\n','')
        
        if (player_info[1] == '.' and player_meta[1] == '.') or player_meta[1] == '.' or (player_info[1] != '.' and player_meta[1] != '.'):
            player = player_info.split(' (ID')[0]
        else:
            player = player_meta
            

        #table com informações do time
        tables = page_content.findAll('table')
        try:
            td = tables[0].findAll('td')
            value = td[2].text.split('€')[1].replace('M','').replace('\n','')
            wage = td[3].text.split('€')[1].replace('K','').replace('\n','')
        except:
            value = 0
            wage = 0
        try:
            team_country = tables[1].findAll('a',title=True)[0].attrs['title']
            team_name = tables[1].findAll('a',title=False)[0].text
        except:
            team_country,team_name = 'No Team','No Team'
    except:
        team_country,team_name, player = 'Link Incorreto','Link Incorreto', 'Link Incorreto'
    return [player,team_country,team_name,value,wage]

In [ ]:
data = pd.read_csv('C:/Users/iagoa/Desktop/IC/Arquivos FIFA/FIFA_CONSOLIDADO2.csv')
data['Team_Country'] = ''
data['Correct_Teams'] = ''
data['Correct_Name'] = ''
data['Wage_K'] = 0
data['Value_M'] = 0
data.Teams = data.Teams.str.replace("[",'').str.replace("]",'').str.replace("'","")

In [ ]:
df_TVersion = pd.read_csv('C:/Users/iagoa/Desktop/IC/Arquivos FIFA/FIFA_PVersion_TVersion.csv')
df_TVersion = df_TVersion[['PlayerID','Team','TeamID']].drop_duplicates()
df_TVersion.columns = ['ID','Teams','TeamID']

In [ ]:
df_collect = pd.merge(data,df_TVersion,how='left',on=['ID','Teams'])

In [ ]:
df_log = pd.DataFrame(columns=['Link'])

In [ ]:
#for idx, row in df_teste.loc[df_teste.Team_Country == '',['Link']].iterrows():
for idx, row in df_collect.loc[df_collect.Team_Country == '',['Link']].iterrows():
    try:
        new_data = collect_data(row[0])
        df_collect.loc[idx, ['Correct_Name','Team_Country','Correct_Teams','Value_M','Wage_K']] = new_data            
    except:
        df_log.loc[len(df_log)] = row
df_log.to_csv('log_FIFA.csv',index=False)

In [ ]:
df_collect.to_csv('FIFA_CONSOLIDADO3.csv',index=False)